In [1]:
import json

# JSON file
f = open ('../data/open_source/Blink1_J22.json', "r")
  
# Reading from file
data = json.loads(f.read())
  
# Iterating through the json
# list
for i in data['data']:
    print(i)
  
# Closing file
f.close()

code
commander
mainBoard
name
releaseDate
sideBoard
type


In [2]:

data['data']['mainBoard'][0]['identifiers']['multiverseId']

'591436'

In [3]:
from collections import Counter

temp_list = []
for card in data['data']['mainBoard']:
    temp_list.append(card['identifiers']['multiverseId'])

Counter(temp_list)

Counter({'591436': 1,
         '589558': 1,
         '591463': 1,
         '591477': 1,
         '589562': 1,
         '591508': 1,
         '591532': 1,
         '591543': 1,
         '591415': 1,
         '597039': 1,
         '591519': 1,
         '591458': 1,
         '597745': 1,
         '596241': 1})

In [4]:
len(data['data']['mainBoard'])

14

# other file testing

In [1]:
import pandas as pd

df = pd.read_csv('../data/all_decks/modern_dataset_12-27-2020.csv')
# df = df[0:100]

In [2]:
df.head()

,deckName,colors,numMain,numSideboard,mainDeck,sideboard
0,Whiteless Scapeshift,{U}{B}{R}{G},60,15,2 Blood Crypt 2 Breeding Pool 1 Bring to Light...,3 Collective Brutality 2 Damnation 1 Damping S...
1,Mr. Burns,{R},60,0,4 Act of Treason 4 Goblin Chainwhirler 4 Infur...,NaN
2,Selesnya Angels,{W}{G},60,0,1 Ajani Mentor of Heroes 1 Angelic Gift 1 Avac...,NaN
3,Undergrowth Menace,{B}{G},60,13,8 Forest 4 Blooming Marsh 4 Temple of Malady 4...,2 The Ozolith 2 Rite of Consumption 3 Grasp of...
4,Cleansing Wildfire,{W}{U}{R},60,15,2 Bloodwater Entity 1 Cascading Cataracts 2 Ce...,2 Aether Gust 2 Blood Moon 1 Chromatic Lantern...


In [3]:
# creating the mainDeckDict

mainDeckDict = [[] for i in range(len(df))]

for idx,main_deck in enumerate(df['mainDeck']):
    cards_list = main_deck.split()
    cards_dict = {}

    count = 0
    card_name = ""

    for entity in cards_list:
        if entity.isdigit():
            cards_dict[card_name[0:-1]] = count
            card_name = ""
            count = int(entity)
            if count > 4:
                count = 4
        else:
            card_name += entity + " "
    cards_dict[card_name[0:-1]] = count
    del cards_dict['']

    # just some data cleaning
    for key,item in cards_dict.items():
        if key in ['Mountain', 'Swamp', 'Island', 'Forest', 'Plains'] and item > 40:
            cards_dict[key] = -1
        elif item > 4:
            cards_dict[key] = -1
        
    mainDeckDict[idx] = cards_dict

df['mainDeckDict'] = mainDeckDict
del mainDeckDict

In [4]:
df.head()

,deckName,colors,numMain,numSideboard,mainDeck,sideboard,mainDeckDict
0,Whiteless Scapeshift,{U}{B}{R}{G},60,15,2 Blood Crypt 2 Breeding Pool 1 Bring to Light...,3 Collective Brutality 2 Damnation 1 Damping S...,"{'Blood Crypt': 2, 'Breeding Pool': 2, 'Bring ..."
1,Mr. Burns,{R},60,0,4 Act of Treason 4 Goblin Chainwhirler 4 Infur...,NaN,"{'Act of Treason': 4, 'Goblin Chainwhirler': 4..."
2,Selesnya Angels,{W}{G},60,0,1 Ajani Mentor of Heroes 1 Angelic Gift 1 Avac...,NaN,"{'Ajani Mentor of Heroes': 1, 'Angelic Gift': ..."
3,Undergrowth Menace,{B}{G},60,13,8 Forest 4 Blooming Marsh 4 Temple of Malady 4...,2 The Ozolith 2 Rite of Consumption 3 Grasp of...,"{'Forest': 4, 'Blooming Marsh': 4, 'Temple of ..."
4,Cleansing Wildfire,{W}{U}{R},60,15,2 Bloodwater Entity 1 Cascading Cataracts 2 Ce...,2 Aether Gust 2 Blood Moon 1 Chromatic Lantern...,"{'Bloodwater Entity': 2, 'Cascading Cataracts'..."


In [5]:
# summary of cards

from collections import Counter
summary_dict = {}
for deck_dict in df['mainDeckDict']:
    summary_dict = Counter(summary_dict) + Counter(deck_dict) #{**summary_dict, **deck_dict} # 
summary_dict = dict(summary_dict)
summary_dict = {k: v for k, v in sorted(summary_dict.items(), key=lambda item: item[1], reverse=True)}

KeyboardInterrupt: 

In [6]:
# main deck list
all_cards = []

for deck in df['mainDeckDict']:
    for card in deck.keys():
        all_cards.append(card)
all_cards = list(set(all_cards))
all_cards.sort()

In [7]:
# deck Vector, takes about 4 mins to run
all_deck_vectors = []
for deck in df['mainDeckDict']:
    deck_vector = [0 for x in range(len(all_cards))]
    for card in deck.keys():
        for idx,card_name in enumerate(all_cards):
            if card == card_name:
                deck_vector[idx] = 1
    all_deck_vectors.append(deck_vector)

df['mainDeckVector'] = all_deck_vectors


In [11]:
df.to_csv(r'../data/all_decks/cleaned_data.csv', index=False)

In [8]:
import numpy as np

def cosine_similarity(a, b):
    """
    Computes cosine similarity between two vectors a and b.
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# Example usage
a = np.array([1, 2, 3])
b = np.array([3, 2, 1])
similarity = cosine_similarity(a, b)
print("Cosine similarity between a and b:", similarity)


Cosine similarity between a and b: 0.7142857142857143


In [12]:
deck_vector_comparison = df['mainDeckVector'][2]

similarity = []

for deck_vector in df['mainDeckVector']:
    similarity.append(cosine_similarity(deck_vector_comparison, deck_vector))

df['cosineSimilarity'] = similarity
df.sort_values(by=['cosineSimilarity'], ascending=False).head(10)

,deckName,colors,numMain,numSideboard,mainDeck,sideboard,mainDeckDict,mainDeckVector,cosineSimilarity
2,Selesnya Angels,{W}{G},60,0,1 Ajani Mentor of Heroes 1 Angelic Gift 1 Avac...,NaN,"{'Ajani Mentor of Heroes': 1, 'Angelic Gift': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.000000
3984,Selesnya Larissa,{W}{G},60,11,8 Forest 2 Plains 4 Temple Garden 4 Windswept ...,3 Once Upon a Time 4 Voice of Resurgence 4 Shi...,"{'Forest': 4, 'Plains': 2, 'Temple Garden': 4,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.404929
5933,Company-Angel,{W}{G},60,15,2 Forest 2 Ghost Quarter 6 Plains 4 Sunpetal G...,2 Grafdigger's Cage 2 Rest in Peace 2 Spirit o...,"{'Forest': 2, 'Ghost Quarter': 2, 'Plains': 4,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.368037
3469,Karma is a ... nice thing,{W}{B}{G},60,0,4 Aura of Silence 4 Birds of Paradise 4 Contam...,NaN,"{'Aura of Silence': 4, 'Birds of Paradise': 4,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.350823
6677,Brisela Voice of Nightmares,{W}{G},62,10,4 Branchloft Pathway // Boulderloft Pathway 2 ...,1 Grafdigger's Cage 1 Damping Sphere 1 Stony S...,{'Branchloft Pathway // Boulderloft Pathway': ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.350823
6675,Ich glaube ich bin Zufrieden @ Engel.,{W}{G},60,13,4 Branchloft Pathway // Boulderloft Pathway 2 ...,1 Grafdigger's Cage 1 Damping Sphere 1 Stony S...,{'Branchloft Pathway // Boulderloft Pathway': ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.342368
8145,Cat Aggro,{W}{G},60,1,4 Brimaz King of Oreskos 2 Kaheera the Orphang...,1 Kaheera the Orphanguard,"{'Brimaz King of Oreskos': 4, 'Kaheera the Orp...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.294174
3013,Aggro Angel Life Gain,{W}{G},60,15,1 Aetherflux Reservoir 3 Angel of Vitality 4 B...,1 Choke 2 Damping Sphere 2 Dawn Charm 1 Gaea's...,"{'Aetherflux Reservoir': 1, 'Angel of Vitality...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.292685
6371,Bant Broodmoth,{W}{U}{G},60,15,4 Phyrexian Unlife 4 Solemnity 4 Uro Titan of ...,2 Nine Lives 2 Dovin's Veto 3 Veil of Summer 2...,"{'Phyrexian Unlife': 4, 'Solemnity': 4, 'Uro T...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.292685
4942,New bant,{W}{U}{G},62,1,4 Birds of Paradise 3 Breeding Pool 2 Brimaz K...,1 Elspeth Sun's Champion,"{'Birds of Paradise': 4, 'Breeding Pool': 3, '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.292685


In [ ]:
deck_vector_comparison = df['mainDeckVector'][2]

similarity = []

for deck_vector in df['mainDeckVector']:
    deck_vector_comparison
    similarity.append(cosine_similarity(deck_vector_comparison, deck_vector))

df['cosineSimilarity'] = similarity
df.sort_values(by=['cosineSimilarity'], ascending=False).head(10)

In [15]:
deck_vector_compressed = deck_vector_comparison

In [19]:
[i for i, e in enumerate(deck_vector_compressed) if e == 1]

[154,
 307,
 583,
 823,
 1037,
 1343,
 1589,
 2721,
 2935,
 3309,
 3551,
 4072,
 4588,
 5223,
 5322,
 5659,
 6487,
 6644,
 7263,
 7808,
 7823,
 7871,
 8770,
 9000,
 10227,
 10289]